# BC Test

Testscript to compare results of neighborlists hydrogen tracing to molecular descriptors.
This script and the accompanying `.py` scripts are crafted to work within the folder structure provided by BC.

In [ ]:
import matplotlib.pyplot as plt
import sys
import numpy as np
from ase.io import read, write
from ase import Atoms
from ase.neighborlist import NeighborList
from scipy.spatial.distance import cdist
from tqdm import tqdm
from matplotlib import cm

from asaplib.reducedim import Dimension_Reducers
from asaplib.plot import Plotters as asapPlotters

from ase.neighborlist import natural_cutoffs, NeighborList
import pandas as pd

In [ ]:
system_list = [
'anatase-100-nd-0',
'anatase-101-nd-0',
'anatase-110-nd-0',
'rutile-001-nd-0',
'rutile-011-nd-0',
'rutile-100-nd-0',#'rutile-101-nd-0',
'rutile-110-nd-0']

In [ ]:
surface_name_dict={"anatase-100":"anatase (100)",
                    "anatase-101":"anatase (101)",
                    "anatase-110": "anatase (110)",
                    "rutile-001": "rutile (001)",
                     "rutile-011": "rutile (011)", 
                    "rutile-100": "rutile (100)",
                    "rutile-110": "rutile (110)"}

In [ ]:
system_now = 'rutile-011-nd-0'

# Possible reshaping issue with 'rutile-100-nd-0', seems to not be 12 long but 11

nhydrogen = 128*2

ts_slicer = np.s_[1000:10000:10]

In [ ]:
system_ref = 'anatase-101-nd-0' # always use this
with open('ref-anatase-101-nd-0-h-dis.npy', 'rb') as f:
    h_dis_all_ref = np.load(f)
    print(h_dis_all_ref.shape)

In [ ]:
colvar = np.genfromtxt(system_now+'/COLVAR')[ts_slicer] # consistent with the gen of features
tt = 330 #K
kbt = 0.008314*tt # in kj/mol

print(len(colvar))
sys_weights = np.exp((colvar[:,2]+colvar[:,3])/kbt)

In [ ]:
with open(system_now+'-h-dis-env.npy', 'rb') as f:
    print(system_now)
    h_dis_all = np.load(f)
    h_env_all = np.load(f)
    print(h_dis_all.shape)
    print(h_env_all.shape)

In [ ]:
h_weights = np.ones((len(sys_weights),nhydrogen))
for i in range(len(sys_weights)):
    h_weights[i,:] *= sys_weights[i]

# print(h_weights.T)

In [ ]:
reduce_dict = {}
"""
reduce_dict['pca'] = {
    "type": 'PCA',
    'parameter':{
        "n_components": 4}
}
"""
reduce_dict['kpca'] = {
    "type": 'SPARSE_KPCA',
    'parameter':{
        "n_components": 2,
        "n_sparse": 200, # no sparsification
        "kernel": {"first_kernel": {"type": 'cosine'}}
    }
}

dreducer = Dimension_Reducers(reduce_dict)

hcoord_ref = np.reshape(h_dis_all_ref,(-1,11))
hcoord_now = np.reshape(h_dis_all,(-1,h_dis_all.shape[-1]))

print(hcoord_ref.shape)
print(hcoord_now.shape)

# proj = dreducer.fit_transform(hcoord_now[:,[1,2,3,5,6,8,9,10]])

In [ ]:
dreducer.fit(hcoord_ref[:,[1,2,3,5,6,8,9,10]])
proj = dreducer.transform(hcoord_now[:,[1,2,3,5,6,8,9,10]])

print(proj.shape)

In [ ]:
# reshape
print(proj.shape[0]/nhydrogen)
h_proj = np.reshape( proj[:,[0,1]],(-1,nhydrogen,2))
print(np.shape(h_proj))

stride = 1
h_proj_sparse = h_proj[::stride,:,:]
np.shape(h_proj_sparse)

h_proj_sparse_all = np.reshape(h_proj_sparse[1:,:,:],(-1,2))

In [ ]:
# classify

cls_labels= ['H$_2$O$^{(> 1)}$','H-O$_t$','HO-Ti','H$_2$O-Ti','H$_2$O$^{(1)}$']

cls = np.zeros(len(hcoord_now))
for i,hh in enumerate(hcoord_now):
    if hh[3] >= 5: 
        cls[i] = 0 # in the bulk
    elif hh[3] < 1.25 and hh[2] > 1.6:
        cls[i] = 1 # H on O(TiO2)
    elif hh[2] > 1.85 and hh[5] < 3: # OH on Ti
        cls[i] = 2 # OH on Ti
    elif hh[5] < 3:
        cls[i] = 3 # H2O on Ti
    else:
        cls[i] = 4 # H2O close to slab but not on Ti
        
cls =  np.reshape( cls,(-1,nhydrogen))

In [ ]:
trajectory = read(system_now+'/out.lammpstrj', index='%u:%u:%u'%(ts_slicer.start, ts_slicer.stop, ts_slicer.step))
cutoffs = natural_cutoffs(trajectory[0], mult=0.75)

cur_nlist = NeighborList(cutoffs, bothways=True, self_interaction=False)
prev_nlist = NeighborList(cutoffs, bothways=True, self_interaction=False)
long_nlist = NeighborList(natural_cutoffs(trajectory[0], H=3, Ti=3, O=3), bothways=True, self_interaction=False)
print(ts_slicer)
print(len(trajectory))

In [ ]:
def get_o_config(nlist, h_index, init_config):
    h_neighs = nlist.get_neighbors(h_index)[0]
    
    if len(h_neighs) == 0:
        return 'solo'
    
    h_neigh = h_neighs[0] # Closest element should be O anyways
    
    o_neighs = nlist.get_neighbors(h_neigh)[0]
    o_neighs = np.append(h_neigh, o_neighs)
    
    return init_config[o_neighs].get_chemical_formula(mode='hill')

def get_o_index(nlist, h_index, snapshot):
    h_neighs = nlist.get_neighbors(h_index)[0]
    h_neighs.sort()
    dists = snapshot.get_distances(h_index, h_neighs, mic=True)
    sort_args = np.argsort(dists)
    dists = dists[sort_args]
    h_neighs = h_neighs[sort_args]
    
    if len(h_neighs) == 0:
        return 'solo'
    
    h_neigh = h_neighs[0] # Closest element should be O anyways
    return h_neigh

def get_nearest_symb(nlist, target_index, snapshot, symb='H'):
    h_neighs = nlist.get_neighbors(target_index)[0]
    # h_neighs.sort()
    dists = snapshot.get_distances(target_index, h_neighs, mic=True)
    sort_args = np.argsort(dists)
    dists = dists[sort_args]
    h_neighs = h_neighs[sort_args]

    for ii_neigh, h_neigh in enumerate(h_neighs):
        if snapshot[h_neigh].symbol == symb:
            return dists[ii_neigh], h_neighs[ii_neigh]

def get_nearest_h(nlist, h_index, config: Atoms, symb='H'):
    h_neighs = nlist.get_neighbors(h_index)[0]
    dists = config.get_distances(h_index, h_neighs)
    sort_args = np.argsort(dists)
    dists = dists[sort_args]
    h_neighs = h_neighs[sort_args]

    for ii_neigh, h_neigh in enumerate(h_neighs):
        if config[h_neigh].symbol == symb:
            return h_neigh

In [ ]:
def update_nlists(i):
    # not okay to write functions like this
    cur_nlist.update(trajectory[i])
    prev_nlist.update(trajectory[i-1])
    long_nlist.update(trajectory[i-1])

# weighted transition rates (assuming V is quasi-static)

start_configs = []
end_configs = []
nearest_hs = []

# wstart = []
# wend = []

cl_transition = np.zeros((5,5))
print(cls.shape)
# I want to plot the evolution of H

count = 0
nl_ts = 0
choice_param = []
for i in range(1,np.shape(cls)[0]): # loop through the frames
    for j in range(nhydrogen):# loop through points 
        ts_configs = []   
        [c1, c2] = [int(cls[i,j]), int(cls[i-1,j])]
        cl_transition[c1,c2] += 1 # h_weights[i,j]/h_weights[i-1,j]
        if (c1 == 1) and (c2 == 4):
            print("timestep: %u"%(1000+(10*i)))
            print("Hydrogen: %u"%int(h_dis_all[i, j, 0]))
            # print(h_dis_all[i-1, j, 2])
            # print(h_dis_all[i-1, j, 3])
            print(h_dis_all[i-1, j, 5])
            choice_param.append(h_dis_all[i-1, j, 5])
        if (c1 == 1) and (c2 == 3):
            count += 1
            
            if not nl_ts == i:
                update_nlists(i)
                nl_ts = i

            # print(f"Timestep: {1000+(10*i):n}")
            # print(f"Hydro Index: {h_dis_all[i, j, 0]:n}")
            # if count == 50:
            #     raise ValueError
            h_index = int(h_dis_all[i, j, 0])

            # end_configs.append(get_o_config(cur_nlist, h_index, trajectory[0]))
            # start_configs.append(get_o_config(prev_nlist, h_index, trajectory[0]))
            # nearest_hs.append(get_nearest_h(long_nlist, h_index, trajectory[i-1]))
            # print("Distance in Hcoord prev")
            # print(h_dis_all[i-1, j, 2])
            # print("Nearest H dist")
            # print(trajectory[i-1].get_distance(h_index, nearest_hs[-1]))

            o_neigh = get_o_index(long_nlist, h_index, trajectory[i-1])
            try:
                ti_dist, ti_ind = get_nearest_symb(long_nlist, o_neigh, trajectory[i-1], symb='Ti')
                if ti_dist > 3 or (not np.allclose(ti_dist, h_dis_all[i-1, j, 5])):
                    print("Ti Distance from Ti %i to O %i: %f"%(ti_ind, o_neigh, ti_dist))
                    print("h_dis: %f"%h_dis_all[i-1, j, 5])
            except:
                print("ONLY H DIS: %f"%h_dis_all[i-1, j, 5])

print(count)
print(cls.shape)
print(cl_transition)
print("Total dissociations: ", cl_transition[1, 3]+cl_transition[1, 4])

for k in range(5):
    cl_norm = np.sum(cl_transition[k,:])
    cl_transition[k,:]/=cl_norm

In [ ]:
choice_param = np.asarray(choice_param)
print(choice_param.shape)
bin_edges, hist = np.histogram(choice_param)
print(bin_edges, hist)
plt.hist(choice_param, bins=10)
plt.savefig(f'bins_{system_now.split("-")[0]}_{system_now.split("-")[1]}.pdf', format='pdf')

In [ ]:
print("Starting configurations:")
start_counts = pd.Series(start_configs).value_counts()
print(start_counts)

print("Final configurations:")
end_counts = pd.Series(end_configs).value_counts()
print(end_counts)

if False:
    print("Starting configurations:")
    counts = pd.Series(wstart).value_counts()
    print(counts)

    print("Final configurations:")
    counts = pd.Series(wend).value_counts()
    print(counts)

In [ ]:
from matplotlib.patches import Patch

fig, axes = plt.subplots(nrows=2, ncols=1)

total_loc = -2
x_start = np.arange(len(start_counts))
x_end = np.arange(len(end_counts))
fs = 15

axes[0].bar(x_start, start_counts.array)
axes[0].bar(total_loc, np.sum(start_counts.array))
axes[0].set_xticks(np.append(total_loc, x_start))
axes[0].set_xticklabels([r'H$_2$O$^{(1)}$'] + start_counts.keys().tolist())
axes[0].annotate(
    "", 
    xy=(total_loc+0.5+1, np.sum(start_counts.array)/2), 
    xytext=(total_loc+0.5, np.sum(start_counts.array)/2), 
    arrowprops=dict(arrowstyle="->")
)
axes[0].set_title("Starting Configurations", fontsize=fs-5)
axes[0].set_ylabel("counts")

axes[1].bar(x_end, end_counts.array)
axes[1].bar(total_loc, np.sum(end_counts.array))
axes[1].set_xticks(np.append(total_loc, x_end))
axes[1].set_xticklabels(['H-O$_t$'] + end_counts.keys().tolist())
axes[1].annotate(
    "", 
    xy=(total_loc+0.5+1, np.sum(end_counts.array)/2), 
    xytext=(total_loc+0.5, np.sum(end_counts.array)/2), 
    arrowprops=dict(arrowstyle="->")
)
axes[1].set_title("End Configurations", fontsize=fs-5)
axes[1].set_ylabel("counts")

legend_elements = [
    Patch(facecolor='tab:orange', label='atomic descriptors classification'),
    Patch(facecolor='tab:blue', label='nearest neighbour environment')
]
axes[0].legend(handles=legend_elements)

fig.suptitle(f'{system_now.split("-")[0]} ({system_now.split("-")[1]})', fontsize=fs)

plt.tight_layout()

fig.savefig(f'advsnl_{system_now.split("-")[0]}_{system_now.split("-")[1]}.pdf', format='pdf')

plt.show()

In [ ]:
%matplotlib inline 

fig = plt.figure(figsize=(3.5,3.5), dpi=300)
ax = plt.gca()

from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

im = ax.matshow(np.log10(cl_transition), vmin=-6, vmax=0)
plt.xticks(np.arange(5), cls_labels)
plt.yticks(np.arange(5), cls_labels)

# one-step
#plt.plot([1],[3],'*', ms=10, c='r')
plt.plot([3],[1],'*', ms=10, c='r')

# one-step
#plt.plot([1,3],[4,4],'^', ms=10, c='cyan')
plt.plot([4,4],[1,3],'^', ms=10, c='cyan')

plt.tick_params(axis='both', which='major', labelsize=8)

plt.title(system_now[:-5], y=1.08)
# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.15)
   
cbar = plt.colorbar(im, cax=cax, )
cbar.set_label('relative transition rates', rotation=90)

fig.tight_layout()

plt.show()

## Plotting

In [ ]:
def get_draw_indices(ret_count=0):
    count = 0
    for i in range(1,np.shape(cls)[0]): # loop through the frames
        for j in range(nhydrogen):# loop through points    
            [c1, c2] = [int(cls[i,j]), int(cls[i-1,j])]
            cl_transition[c1,c2] += h_weights[i,j]/h_weights[i-1,j]
            if (c1 == 1) and (c2 == 4):
                count += 1
            if count == ret_count:
                return i, int(h_dis_all[i, j, 0])

def get_o_surroundings(nlist, h_index):
    h_neighs = nlist.get_neighbors(h_index)[0]
    
    if len(h_neighs) == 0:
        return 'solo'
    
    h_neigh = h_neighs[0] # Closest element should be O anyways
    
    o_neighs = nlist.get_neighbors(h_neigh)[0]
    o_neighs = np.append(h_neigh, o_neighs)
    
    return o_neighs

def shift_snapshot(snapshot: Atoms):
    # Shift all positions half a unit cell
    at_pos = snapshot.get_positions()
    at_pos[:, :2] = 0.
    at_pos[:, 2] = snapshot.cell[2][2]*0.5
    snapshot.translate(at_pos)
    snapshot.wrap()
    return snapshot

In [ ]:
nlist_surr = False
zmax = 6
zmin = 2

plot_ts, h_index = get_draw_indices(ret_count=10)
print(plot_ts, h_index)

plot_cutoffs = natural_cutoffs(trajectory[0], mult=4)
plot_nl = NeighborList(plot_cutoffs, bothways=True, self_interaction=False)

plot_nl.update(trajectory[plot_ts])
cur_nlist.update(trajectory[plot_ts])
if nlist_surr:
    cur_surrounding = get_o_surroundings(plot_nl, h_index)
else:
    shifted_snapshot = shift_snapshot(trajectory[plot_ts][:])
    zs = shifted_snapshot.get_positions()[:, -1]
    cur_surrounding = np.argwhere(np.logical_and(zs > (zs[h_index]-zmin), zs < (zs[h_index]+zmax))).squeeze()

cur_neighs = get_o_surroundings(cur_nlist, h_index)
cur_nearest_h_ind = get_nearest_h(plot_nl, h_index=h_index, config=trajectory[plot_ts])

plot_nl.update(trajectory[plot_ts-1])
if nlist_surr:
    prev_surrounding = get_o_surroundings(plot_nl, h_index)
else:
    shifted_snapshot = shift_snapshot(trajectory[plot_ts-1][:])
    all_pos = shifted_snapshot.get_positions()
    xs = all_pos[:, 0]
    ys = all_pos[:, 1]
    zs = all_pos[:, 2]
    x_cond = np.logical_and(xs > (xs[h_index]-5), xs < (xs[h_index]+5))
    y_cond = np.logical_and(ys > (ys[h_index]-5), ys < (ys[h_index]+5))
    z_cond = np.logical_and(zs > (zs[h_index]-zmin), zs < (zs[h_index]+zmax))
    prev_surrounding = np.logical_and(np.logical_and(x_cond, y_cond), z_cond)

cur_nlist.update(trajectory[plot_ts-1])
prev_neighs = get_o_surroundings(cur_nlist, h_index)
prev_nearest_h_ind = get_nearest_h(plot_nl, h_index=h_index, config=trajectory[plot_ts-1])

print(cur_neighs)
print(prev_neighs)

In [ ]:
print(prev_surrounding.shape)

In [ ]:
%matplotlib auto
c_dict = {"H": "b", "O": "r", "Ti": "gray"}

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection='3d')
ax.set_title("End Configuration")

in_closest = []
for ii_neigh, neigh in enumerate(cur_surrounding):
    if neigh in cur_neighs:
        in_closest.append(ii_neigh)
    if neigh == cur_nearest_h_ind:
        nearest_ind = ii_neigh

cur_snapshot = shift_snapshot(trajectory[plot_ts][:])
all_pos = cur_snapshot.get_positions()
cur_snapshot = cur_snapshot[cur_surrounding]

# Get colors
at_symbs = cur_snapshot.get_chemical_symbols()
colors = []
for at_symb in at_symbs:
    colors.append(c_dict[at_symb])

for ii_close in in_closest:
    colors[ii_close] = "y"
colors[nearest_ind] = 'g'

at_pos = cur_snapshot.get_positions()

sizes = np.array(natural_cutoffs(cur_snapshot, mult=500))

sc = ax.scatter(
    at_pos[:, 0], at_pos[:, 1], at_pos[:, 2],
    s=sizes,
    c=colors,
    alpha=1.,
    edgecolors="k", # vmin=0, vmax=1
)
print(shift_snapshot(trajectory[plot_ts][:]).get_distance(h_index, cur_nearest_h_ind))
ax.plot(
    [all_pos[h_index, 0], all_pos[cur_nearest_h_ind, 0]],
    [all_pos[h_index, 1], all_pos[cur_nearest_h_ind, 1]],
    [all_pos[h_index, 2], all_pos[cur_nearest_h_ind, 2]],
    linewidth=3, linestyle='-', color='k'
)

z_span = (ax.get_zlim()[1] - ax.get_zlim()[0])/2.
ax.set_ylim([np.mean(ax.get_ylim()) - z_span, np.mean(ax.get_ylim()) + z_span])
ax.set_xlim([np.mean(ax.get_xlim()) - z_span, np.mean(ax.get_xlim()) + z_span])
# cbar = fig.colorbar(sc)
plt.show()

In [ ]:
%matplotlib auto
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection='3d')
ax.set_title("Starting Configuration")

if False:
    prev_surrounding = np.array([
        460, 5, # Hydrogen and Oxygen
        3, 0, # Surface Ti
        459, 461, 271 # nearest H20
    ] ,dtype=np.int32)
in_closest = []
for ii_neigh, neigh in enumerate(prev_surrounding):
    if neigh in prev_neighs:
        in_closest.append(ii_neigh)
    if neigh == prev_nearest_h_ind:
        nearest_ind = ii_neigh
        
cur_snapshot = shift_snapshot(trajectory[plot_ts-1][:])
all_pos = cur_snapshot.get_positions()
cur_snapshot = cur_snapshot[prev_surrounding]

print("TS: %u"%(1000+(plot_ts-1)*10))
print("h index: %u, nearest_neigh: %u"%(h_index, prev_nearest_h_ind))

# Get colors
at_symbs = cur_snapshot.get_chemical_symbols()
colors = []
for at_symb in at_symbs:
    colors.append(c_dict[at_symb])

if False:
    for ii_close in in_closest:
        colors[ii_close] = "y"
    colors[nearest_ind] = 'g'

at_pos = cur_snapshot.get_positions()

sizes = np.array(natural_cutoffs(cur_snapshot, mult=6000))

sc = ax.scatter(
    at_pos[:, 0], at_pos[:, 1], at_pos[:, 2],
    s=sizes,
    c=colors,
    alpha=1,
    edgecolors="k", # vmin=0, vmax=1
    zorder=1
)
print(shift_snapshot(trajectory[plot_ts][:]).get_distance(h_index, prev_nearest_h_ind))
ax.plot(
    [all_pos[h_index, 0], all_pos[prev_nearest_h_ind, 0]],
    [all_pos[h_index, 1], all_pos[prev_nearest_h_ind, 1]],
    [all_pos[h_index, 2], all_pos[prev_nearest_h_ind, 2]],
    linewidth=3, linestyle='-', color='k', zorder=2
)

z_span = (ax.get_zlim()[1] - ax.get_zlim()[0])/2.
ax.set_ylim([np.mean(ax.get_ylim()) - z_span, np.mean(ax.get_ylim()) + z_span])
ax.set_xlim([np.mean(ax.get_xlim()) - z_span, np.mean(ax.get_xlim()) + z_span])
# cbar = fig.colorbar(sc)
plt.show()

## All configurations

In [ ]:
all_configs = []

for i in range(1,np.shape(cls)[0]): # loop through the frames
    cur_nlist.update(trajectory[i])
    # prev_nlist.update(trajectory[i-1])
    
    for j in range(nhydrogen):# loop through points
        h_index = int(h_dis_all[i, j, 0])
        all_configs.append(get_o_config(cur_nlist, h_index, trajectory[0]))

In [ ]:
all_counts = pd.Series(all_configs).value_counts()
print("Found configurations: ")
print(all_counts)

# Which index to give which entry in config
classification_dict = {'H2O':0, 'H2OTi':1, 'HOTi':2, 'HOTi2':3, 'H3O':0, 'HO':2, 'H2OTi2':3}
nl_classi = np.zeros((cls.shape[0]*cls.shape[1],), dtype=np.int8)
for ii_conf, config in enumerate(all_configs):
    nl_classi[ii_conf] = classification_dict.get(config, -1)

nl_classi = nl_classi.reshape(cls.shape)
assert np.all(nl_classi>=0), "Found undefined config"

### Plotting

In [ ]:
%matplotlib inline

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))

classes = ['H2O', 'H2OTi', 'HOTi', 'HOTi2']
n_classes = len(classes)

cmap = cm.get_cmap('tab10', n_classes)
sc = axes[0].scatter(proj[::1,0], proj[::1,1], c=nl_classi, cmap=cmap, s=2, vmin=-0.5, vmax=n_classes-0.5, rasterized=True)
#cbaxes = ax.inset_axes([0.0, 1.1, 1.0, 0.02])``
#cbar= plt.colorbar(sc, cax=cbaxes, ticks=[0, 1, 2, 3, 4], orientation='horizontal')
cbar = plt.colorbar(sc, ax=axes[0], ticks=np.arange(n_classes))
cbar.ax.set_yticklabels(classes)
#cbar.set_label('Classification',labelpad=-3) #, rotation=90)
cbar.set_label('Classification', rotation=90)
axes[0].set_title("Neighbourlist Classification")

plotcolor = cls
cmap = cm.get_cmap('jet', 5)
sc = axes[1].scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap=cmap, s=2, vmin=-0.5, vmax=4.5, rasterized=True)
#cbaxes = ax.inset_axes([0.0, 1.1, 1.0, 0.02])
#cbar= plt.colorbar(sc, cax=cbaxes, ticks=[0, 1, 2, 3, 4], orientation='horizontal')
cbar = plt.colorbar(sc, ax=axes[1], ticks=[0, 1, 2, 3, 4])
cbar.ax.set_yticklabels(cls_labels)
#cbar.set_label('Classification',labelpad=-3) #, rotation=90)
cbar.set_label('Classification', rotation=90)
axes[1].set_title("Atomic Environments Classification")

fig.suptitle(f'PCA map with Neighbourlist Classification vs. Atomic Environments Classification\n{system_now.split("-")[0]} ({system_now.split("-")[1]})')

plt.tight_layout()
# fig.savefig(f'advsnl_pca_{system_now.split("-")[0]}_{system_now.split("-")[1]}.pdf', format='pdf', dpi=300)
plt.show()